In [1]:
from pycaret.datasets import get_data
from pycaret.classification import *
import pandas as pd
from pandas_profiling import ProfileReport


data = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")
Dataset = data.iloc[:, 1:]
Dataset

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...
369,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


In [2]:
Dataset.dtypes

Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
Blood Pressure              object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder              object
dtype: object

In [3]:
# Pisahkan kolom blood_pressure menjadi dua kolom: systole dan diastole
Dataset[['systole', 'diastole']] = Dataset['Blood Pressure'].str.split('/', expand=True)

# Konversi kolom systole dan diastole ke tipe data numerik
Dataset['systole'] = pd.to_numeric(Dataset['systole'])
Dataset['diastole'] = pd.to_numeric(Dataset['diastole'])

# Hapus kolom asli blood_pressure jika tidak diperlukan lagi
Dataset = Dataset.drop(columns=['Blood Pressure'])

Dataset

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,systole,diastole
0,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,NaN,126,83
1,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,NaN,125,80
2,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,NaN,125,80
3,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
4,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
370,Female,59,Nurse,8.0,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
371,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
372,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95


In [4]:
Dataset = Dataset.dropna(subset=['Sleep Disorder'])
s = setup(Dataset, target = 'Sleep Disorder', session_id = 123, 
        categorical_features = ['Gender', 'BMI Category'], ignore_features=['Occupation'])

,Description,Value
0,Session id,123
1,Target,Sleep Disorder
2,Target type,Binary
3,Target mapping,"Insomnia: 0, Sleep Apnea: 1"
4,Original data shape,"(155, 13)"
5,Transformed data shape,"(155, 15)"
6,Transformed train set shape,"(108, 15)"
7,Transformed test set shape,"(47, 15)"
8,Ignore features,1
9,Numeric features,9


In [5]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9091,0.0000,0.9091,0.9197,0.9081,0.8166,0.8271,0.9600
ridge,Ridge Classifier,0.8982,0.0000,0.8982,0.9136,0.8970,0.7971,0.8111,0.0370
lda,Linear Discriminant Analysis,0.8982,0.0000,0.8982,0.9136,0.8970,0.7971,0.8111,0.0280
nb,Naive Bayes,0.8891,0.0000,0.8891,0.9103,0.8870,0.7784,0.7982,0.0410
lightgbm,Light Gradient Boosting Machine,0.8891,0.0000,0.8891,0.9061,0.8880,0.7791,0.7945,0.0460
rf,Random Forest Classifier,0.8800,0.0000,0.8800,0.9025,0.8777,0.7597,0.7810,0.1130
et,Extra Trees Classifier,0.8800,0.0000,0.8800,0.9082,0.8776,0.7609,0.7872,0.0700
catboost,CatBoost Classifier,0.8800,0.0000,0.8800,0.9010,0.8787,0.7616,0.7806,0.4940
ada,Ada Boost Classifier,0.8700,0.0000,0.8700,0.8866,0.8686,0.7428,0.7570,0.0730
dt,Decision Tree Classifier,0.8518,0.0000,0.8518,0.8697,0.8502,0.7037,0.7202,0.0350


In [6]:
model = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9091,0.0000,0.9091,0.9242,0.9091,0.8197,0.8333
1,0.9091,0.0000,0.9091,0.9242,0.9091,0.8197,0.8333
2,0.8182,0.0000,0.8182,0.8636,0.8084,0.6207,0.6708
3,0.9091,0.0000,0.9091,0.9242,0.9091,0.8197,0.8333
4,0.9091,0.0000,0.9091,0.9242,0.9091,0.8197,0.8333
5,0.8182,0.0000,0.8182,0.8182,0.8182,0.6333,0.6333
6,0.8182,0.0000,0.8182,0.8182,0.8182,0.6333,0.6333
7,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [7]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [8]:
finalize_model(model)

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['Age', 'Sleep Duration',
                                             'Quality of Sleep',
                                             'Physical Activity Level',
                                             'Stress Level', 'Heart Rate',
                                             'Daily Steps', 'systole',
                                             'diastole'],
                                    transformer=Simp...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                ('actual_estimator',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=123,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [9]:
save_model(model, 'SleepHealth_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Age', 'Sleep Duration',
                                              'Quality of Sleep',
                                              'Physical Activity Level',
                                              'Stress Level', 'Heart Rate',
                                              'Daily Steps', 'systole',
                                              'diastole'],
                                     transformer=Simp...
                  TransformerWrapper(exclude=None, include=None,
                                     transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                 ('trained_model',
   